In [1]:
import pandas as pd


In [2]:
from datasets import load_dataset

dataset = load_dataset("empathetic_dialogues")


Found cached dataset empathetic_dialogues (/Users/madhavappaneni/.cache/huggingface/datasets/empathetic_dialogues/default/0.1.0/09bbeed3882a67db98c73952fb3c1c9a85af83dc78f81454c2454382fd03f6cf)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
raw_train_data = dataset['train']
train_data = pd.DataFrame(raw_train_data)
raw_test_data = dataset['test']
test_data = pd.DataFrame(raw_test_data)
raw_val_data = dataset['validation']
val_data = pd.DataFrame(raw_val_data)


In [4]:
train_data['utterance'] = train_data['utterance'].str.replace("_comma_", ",")
val_data['utterance'] = val_data['utterance'].str.replace("_comma_", ",")
test_data['utterance'] = test_data['utterance'].str.replace("_comma_", ",")


In [5]:

def preprocess_data(df):
    data_upd = []
    history = []
    max_hist_len = 3
    for i in range(1, len(df)):
        cparts = df.iloc[i - 1]
        sparts = df.iloc[i]
        if cparts["conv_id"] == sparts["conv_id"]:
            prevsent = cparts["utterance"]
            history.append(prevsent)
            idx = int(sparts["utterance_idx"])
            if ((idx % 2) == 0):
                context = " SOC ".join(history[-max_hist_len:])
                # context = ""
                # for string in history[-max_hist_len:]:
                #     context +=  string
                # context = "<|endoftext|>" + context + "<|endoftext|> "
                # label = "<|endoftext|> " + sparts["utterance"] + " <|endoftext|>"
                label = sparts["utterance"]
                data_upd.append(
                    {'conversation': {'input_text': context, 'label': label}})

        else:
            history = []
    return pd.DataFrame(data=data_upd, columns=["conversation"])


In [6]:
preprocess_train = preprocess_data(train_data)
preprocess_test = preprocess_data(test_data)
preprocess_val = preprocess_data(val_data)


In [7]:
preprocess_train.to_pickle('./datasets/train.pkl')
preprocess_test.to_pickle('./datasets/test.pkl')
preprocess_val.to_pickle('./datasets/val.pkl')


In [8]:
print(preprocess_train.sample(5).to_string())


                                                                                                                                                                                                                                                                                                                                                             conversation
20446                                                {'input_text': 'I was shocked. I was at the state park walking through the woods and I swear I saw a Bigfoot. SOC I would have been so excited lol.  What did it look like?! SOC Just like you think it would.. big, tall, hairy walking ape sorta thing', 'label': 'Was anyone with you?  Did anyone believe you?'}
24714                                                                                                                     {'input_text': 'My neighbor returned my wallet today. SOC Where did you lose it at? SOC I lost it at the park. I am so thankful that he found it.', 'label